In [17]:
import sys  
sys.path.insert(1, '.\..\..\SOURCES\.')

import random

random.seed(10)

In [18]:
from DATA import InubushiFunction
from Configuartions import IPReservoirConfiguration, ReservoirConfiguration
from Evaluator import Evaluator
from Metrics import *
from IPMask import IPMask
import itertools

N_UNITS_LIST = [50, 100, 150, 200, 250]


In [19]:
tau_range = [1,2,5,10]

In [20]:
nu_range = np.exp([-2, -1, -0.5, 0, 0.5])
np.log(nu_range)

array([-2. , -1. , -0.5,  0. ,  0.5])

In [21]:
nu_range

array([0.13533528, 0.36787944, 0.60653066, 1.        , 1.64872127])

In [22]:
def evaluate_with_lambda_search(model: Reservoir, data: TimeseriesDATA, metrics: list[Metric]= [MSE()], transient = 100, lambda_range = [0, 0.05, 0.1, 0.15, 0.2]): 
    data.split([80, 10, 10])
    X_TR, Y_TR = data.TR()
    X_VAL, Y_VAL = data.VAL()

    best_lambda=0
    best_err = None
    metric = metrics[0]
    for lambda_thikonov in lambda_range:
        esn = EchoStateNetwork(model)
        esn.train(X_TR, Y=Y_TR, transient=transient, lambda_thikonov=lambda_thikonov)
        error = metric.evaluate(esn.predict(X_VAL), Y_VAL)
        if  best_err == None or best_err > error :
            best_err = error
            best_lambda = lambda_thikonov

    data.split([90,0,10])
    X_TR, Y_TR = data.TR()
    X_TS, Y_TS = data.TS()

    esn = EchoStateNetwork(model)
    esn.train(X_TR, Y=Y_TR, transient=transient, lambda_thikonov=best_lambda)
    y_pred = esn.predict(X_TS)
    return list(np.array([metric.evaluate(y_pred, Y_TS) for metric in metrics])), best_lambda


In [23]:
repetitions = 5

In [24]:
data = InubushiFunction(nu=1)
data.delay_timeseries(1)
data.split([80,0,20])   

X_TR_INIT, _ = data.TR()


mask_names = ["Vanilla", "Gaussian", "Hard Bimodal", "Soft Trimodal", "Edgy trimodal" ]

intrinsic_metrics: list[IntrinsicMetric] = [DeltaPhi(), MC(), MLLE() ,Neff(), Rho(), KL(X_TR_INIT[0:2000])]
estrinsic_metrics: list[EstrinsicMetric] = [MSE(), NRMSE()]
models: list[Reservoir] = []
lambda_range = [0, 0.05, 0.1, 0.15, 0.2]

intrinsic_column_names =  ["Name", "N", "Model Index"] + [metric.name for metric in intrinsic_metrics]
intrinsic_results = []

estrinsic_column_names = ["Name", "N", "Model Index", "Log Nu", "Tau", "Lambda"] +  [metric.name for metric in estrinsic_metrics]
estrinsic_results=[]

for r in range(repetitions):
    print("Repetiotion:", r)
    for N in N_UNITS_LIST:
        init_point = ReservoirConfiguration(N=N ,desired_rho=0.75, input_scaling=0.5, bias=True, bh_scaling=0.1, bu_scaling=0.1,  name=f"Vanilla") 
        model = init_point.build_up_model()
        partial_intrinsic_results = [mask_names[0], N, 0] + list(np.array([metric.evaluate(model) for metric in intrinsic_metrics]))
        intrinsic_results.append(partial_intrinsic_results)

        for nu,tau in itertools.product(*[nu_range, tau_range]): 
            data = InubushiFunction(nu=nu, tau=tau) 
            partial_estrinsic_results, best_lambda = evaluate_with_lambda_search(model, data, estrinsic_metrics, lambda_range=lambda_range)
            estrinsic_results.append([mask_names[0], N, 0, np.log(nu), tau ,best_lambda] + partial_estrinsic_results)
        print("Vanilla Evaluated!")

        masks = [
            IPMask.gaussian(N),
            IPMask.bimodal(N, mu=0.85, std=0.15),
            IPMask.trimodal(N, linear_rate=0.4, std_bim=0.18),
            IPMask.trimodal(N, linear_rate=0.2, mu=0.85, std_bim=0.1)
            ]
        
        for m, mask in enumerate(masks):
            config = IPReservoirConfiguration(init_point, mask=mask, epochs=10, name=f"{mask_names[m+1]}")
            model = config.build_up_model(X_TR_INIT)

            partial_intrinsic_results = [mask_names[m+1], N, m+1] + list(np.array([metric.evaluate(model) for metric in intrinsic_metrics]))
            intrinsic_results.append(partial_intrinsic_results)

            for nu,tau in itertools.product(*[nu_range, tau_range]): 
                data = InubushiFunction(nu=nu, tau=tau)
                partial_estrinsic_results, best_lambda = evaluate_with_lambda_search(model, data, estrinsic_metrics, lambda_range=lambda_range)
                estrinsic_results.append([mask_names[m+1], N, m+1, np.log(nu),tau ,best_lambda] + partial_estrinsic_results)
            print(mask_names[m+1], " Evaluated")

intrinsic_df = pd.DataFrame(intrinsic_results,columns=intrinsic_column_names) #groupby(by=["Name", "Model Index", "N"])
intrinsic_df.to_csv("Inubushi-Intrinsic.csv")

estrinsic_df = pd.DataFrame(estrinsic_results,columns=estrinsic_column_names) #groupby(by=["Name", "Model Index", "N", "Log Nu", "Tau", ])
estrinsic_df.to_csv("Inubushi-Estrinsic.csv")

Repetiotion: 0
Vanilla Evaluated!
Gaussian  Evaluated
Hard Bimodal  Evaluated
Soft Trimodal  Evaluated
Edgy trimodal  Evaluated
Vanilla Evaluated!
Gaussian  Evaluated
Hard Bimodal  Evaluated
Soft Trimodal  Evaluated
Edgy trimodal  Evaluated
Vanilla Evaluated!
Gaussian  Evaluated
Hard Bimodal  Evaluated
Soft Trimodal  Evaluated
Edgy trimodal  Evaluated
Vanilla Evaluated!
Gaussian  Evaluated
Hard Bimodal  Evaluated
Soft Trimodal  Evaluated
Edgy trimodal  Evaluated
Vanilla Evaluated!
Gaussian  Evaluated
Hard Bimodal  Evaluated
Soft Trimodal  Evaluated
Edgy trimodal  Evaluated
Repetiotion: 1
Vanilla Evaluated!
Gaussian  Evaluated
Hard Bimodal  Evaluated
Soft Trimodal  Evaluated
Edgy trimodal  Evaluated
Vanilla Evaluated!
Gaussian  Evaluated
Hard Bimodal  Evaluated
Soft Trimodal  Evaluated
Edgy trimodal  Evaluated
Vanilla Evaluated!
Gaussian  Evaluated
Hard Bimodal  Evaluated
Soft Trimodal  Evaluated
Edgy trimodal  Evaluated
Vanilla Evaluated!
Gaussian  Evaluated
Hard Bimodal  Evaluated
Sof

In [25]:
intrinsic_df.groupby(by=["Name", "Model Index", "N"]).mean()

DeltaPhi         MC      MLLE      Neff  \
Name          Model Index N                                              
Edgy trimodal 4           50   0.090807   8.370542 -1.202819  1.386726   
                          100  0.056274   9.390429 -0.891567  1.381604   
                          150  0.076963  10.192328 -1.219973  1.449068   
                          200  0.088587  10.848763 -0.757170  1.393640   
                          250  0.104752  11.029971 -0.788465  1.437670   
Gaussian      1           50   0.051630  13.945307 -0.152299  2.955285   
                          100  0.048574  17.076636 -0.167966  3.405404   
                          150  0.049639  18.131290 -0.168893  3.602307   
                          200  0.047424  19.569009 -0.160185  3.414881   
                          250  0.054126  20.353489 -0.160880  3.594880   
Hard Bimodal  2           50   0.111652   7.590949 -0.937566  1.274974   
                          100  0.173026   8.921675 -0.908557  1.286657   
                          150  0.096068   9.561316 -0.856479  1.307921   
                          200  0.097874   9.933377 -1.112743  1.264640   
                          250  0.130412  10.353816 -0.913711  1.284056   
Soft Trimodal 3           50   0.064445  10.135239 -0.423609  1.931519   
                          100  0.129518  11.746713 -0.688831  1.863535   
                          150  0.084046  12.917210 -0.411336  2.157901   
                          200  0.087028  12.982629 -0.386776  2.069817   
                          250  0.104183  13.578064 -0.551091  2.106675   
Vanilla       0           50   0.051292  14.180008 -0.334346  2.854937   
                          100  0.052251  16.033047 -0.332515  2.694091   
                          150  0.054719  17.105592 -0.328949  2.763809   
                          200  0.052786  18.820615 -0.313485  2.904161   
                          250  0.048881  19.057777 -0.318135  2.717779   

                                    Rho        KL  
Name          Model Index N                        
Edgy trimodal 4           50   0.819861  0.011478  
                          100  0.819861  0.011521  
                          150  0.819862  0.010968  
                          200  0.819860  0.011337  
                          250  0.819860  0.011181  
Gaussian      1           50   0.819861  0.050098  
                          100  0.819862  0.050669  
                          150  0.819860  0.053354  
                          200  0.819861  0.051839  
                          250  0.819861  0.052521  
Hard Bimodal  2           50   0.819860  0.018051  
                          100  0.819860  0.018400  
                          150  0.819860  0.018315  
                          200  0.819861  0.018389  
                          250  0.819860  0.018622  
Soft Trimodal 3           50   0.819861  0.027911  
                          100  0.819862  0.027931  
                          150  0.819860  0.028999  
                          200  0.819860  0.028755  
                          250  0.819862  0.029752  
Vanilla       0           50   0.750000  0.000000  
                          100  0.750000  0.000000  
                          150  0.750000  0.000000  
                          200  0.750000  0.000000  
                          250  0.750000  0.000000

In [26]:
estrinsic_df.groupby(by=["Name", "Model Index", "N", "Log Nu", "Tau", ]).mean()

Lambda           MSE      NRMSE
Name          Model Index N   Log Nu Tau                                 
Edgy trimodal 4           50  -2.0   1       0.0  7.027351e-09   0.028092
                                     2       0.0  2.226569e-07   0.160995
                                     5       0.0  3.706508e-05   2.046868
                                     10      0.0  3.755762e-03  21.648489
                              -1.0   1       0.0  3.677719e-08   0.024876
...                                          ...           ...        ...
Vanilla       0           250  0.0   10      0.0  6.023781e-04   1.265531
                               0.5   1       0.0  8.688537e-06   0.100469
                                     2       0.0  1.381021e-05   0.127794
                                     5       0.0  2.319949e-04   0.529797
                                     10      0.0  2.809658e-03   1.842730

[500 rows x 3 columns]